## evalution on rank prediction by lap time 

### The Problem

Rank is calculated by the elapsed time when car crosses the start-finish line. The order of the cars for the same lap number is its rank.

This experiment evaluates the performance of a model which predicts the lap time of the next lap.





In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

%load_ext autoreload
%autoreload 2

from indycar.notebook import *

Using TensorFlow backend.


## Load Data

In [3]:
import os
os.getcwd()


'/scratch/hpda/indycar/predictor/notebook/CalculateRank'

In [4]:
D=1

#datalist = ['2017-all_completed_laps_diff.csv','2018-all_completed_laps_diff.csv' ]
datalist = ['2017-Indy500-completed_laps_diff.csv','2018-Indy500-completed_laps_diff.csv' ]
#datalist = read_list('train_all.list')
#datalist = read_list('train_completed.list')

output_prefix = 'indy500-2018-completed_laps-rank'

scaler, dataset, dblens = load_data(datalist)

dataset.info(verbose=True)
print('dataset shape', dataset.shape)
# generate train/test dataset

# post-padding with 0
#X, y, w = generate_data(dataset, D=D, target='time')
X, y, w = generate_data(dataset, D=D, target='time')

total = np.sum(dblens)
ntrain = np.sum(dblens[:-1])
nval = int(dblens[-1] / 2)
print('train=%d, val=%d, test=%d'%(ntrain, nval, total-ntrain-nval))

X_train, X_val, X_test = X[:ntrain], X[ntrain:ntrain + nval], X[ntrain + nval:]
y_train, y_val, y_test = y[:ntrain], y[ntrain:ntrain + nval], y[ntrain + nval:]
#weights
w_train, w_val, w_test = w[:ntrain], w[ntrain:ntrain + nval], w[ntrain+nval:]
print('train shape:', X_train.shape)

load 2017-Indy500-completed_laps_diff.csv, len=3216
load 2018-Indy500-completed_laps_diff.csv, len=3618
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6834 entries, 0 to 3617
Data columns (total 10 columns):
MyIdx             6834 non-null int64
car_number        6834 non-null int64
completed_laps    6834 non-null int64
rank              6834 non-null int64
elapsed_time      6834 non-null float64
rank_diff         6834 non-null float64
time_diff         6834 non-null float64
dbid              6834 non-null int64
rank_diff_raw     6834 non-null float64
time_diff_raw     6834 non-null float64
dtypes: float64(5), int64(5)
memory usage: 587.3 KB
dataset shape (6834, 10)
carNumber = 34, max T =201
train=16, val=9, test=9
train shape: (16, 200, 1)


In [5]:
#
# parameters
#
#year = '2017'
year = '2018'
#event = 'Toronto'
event = 'indy500'
#'indy500-2018-completed_laps'
inputfile = event +'-' + year + '-completed_laps-pred.csv'
outputprefix = year +'-' + event + '-eval-'
pred = pd.read_csv(inputfile)


In [6]:
#rankdiff, timediff
pred_timediff = pred['pred']
print('pred shape',pred_timediff.shape)
pred_timediff[:4]

pred shape (1800,)


0    13.519837
1    34.504074
2    42.128110
3    43.503527
Name: pred, dtype: float64

In [7]:
n = pred_timediff.shape[0]

cars = []
groups = []
for car, group in dataset.groupby('car_number'):
    cars.append(car)
    groups.append(groups)

testset = cars[ntrain+nval:]
rankdata = dataset[dataset['car_number'].isin(testset)]
carnum = len(testset)
print('testset car number', carnum)
    
#print('testset shape', testset.shape)
#groups[ntrain+nval:]

#carnumbers=set()
#rankdata = dataset[-n:]
print('rankdata shape', rankdata.shape)
rankdata[:4]

# reshape to <carno, data>
true_elapsedtime = np.array(rankdata['elapsed_time']).reshape((carnum, -1))
pred_laptime = np.array(pred_timediff).reshape((carnum, -1))


testset car number 9
rankdata shape (1809, 10)


In [8]:
# calculate rank
pred_elapsedtime = np.zeros_like(true_elapsedtime)
pred_elapsedtime[:,1:] = true_elapsedtime[:,0:-1]
pred_elapsedtime[:,1:] += pred_laptime

print(true_elapsedtime[0:2,:10])
print(pred_elapsedtime[0:2,:10])

[[  3.746   48.7191  91.5864 133.604  175.6    218.2643 259.9555 301.6315
  343.4915 385.7495]
 [  4.5005  49.7731  94.0066 136.8631 178.8544 220.6823 262.7334 304.7834
  346.7412 388.8808]]
[[  0.          17.26583725  83.22317407 133.71450951 177.10752671
  218.55476863 260.75466708 301.42276706 342.69711784 384.62290794]
 [  0.          18.02033725  84.41342218 136.845663   181.04056763
  221.99963327 262.72852831 304.26394874 346.09242374 387.9832787 ]]


In [9]:
np.array(rankdata[rankdata['completed_laps']<7]['rank']).reshape((9,-1))

array([[23, 22, 21, 21, 21, 21, 21],
       [32, 28, 27, 27, 26, 26, 26],
       [14, 11, 11, 11, 11, 11, 11],
       [21, 20, 20, 20, 19, 19, 19],
       [31, 32, 31, 30, 31, 31, 31],
       [26, 27, 24, 24, 24, 24, 24],
       [27, 26, 28, 28, 28, 28, 28],
       [22, 23, 22, 22, 22, 22, 22],
       [12, 10, 10, 10, 10, 10, 10]])

In [10]:
# evalutate rank after 5 laps
idx = np.argsort(true_elapsedtime, axis=0)
true_rank = np.argsort(idx, axis=0)
idx = np.argsort(pred_elapsedtime, axis=0)
pred_rank = np.argsort(idx, axis=0)

In [11]:
# simple model
simple_rank = np.zeros_like(true_rank)
simple_rank[:,1:] = true_rank[:,0:-1] 
simple_rank[:,28:35]

array([[3, 3, 3, 3, 3, 2, 1],
       [5, 5, 5, 5, 5, 3, 2],
       [1, 1, 1, 1, 1, 4, 6],
       [2, 2, 2, 2, 2, 1, 0],
       [8, 8, 8, 7, 7, 6, 4],
       [6, 6, 6, 6, 6, 5, 3],
       [7, 7, 7, 8, 8, 7, 8],
       [4, 4, 4, 4, 4, 8, 7],
       [0, 0, 0, 0, 0, 0, 5]])

In [12]:
true_rank[:,28:35]

array([[3, 3, 3, 3, 2, 1, 3],
       [5, 5, 5, 5, 3, 2, 4],
       [1, 1, 1, 1, 4, 6, 1],
       [2, 2, 2, 2, 1, 0, 2],
       [8, 8, 7, 7, 6, 4, 6],
       [6, 6, 6, 6, 5, 3, 5],
       [7, 7, 8, 8, 7, 8, 8],
       [4, 4, 4, 4, 8, 7, 7],
       [0, 0, 0, 0, 0, 5, 0]])

In [13]:
pred_rank[:,28:35]

array([[3, 3, 3, 3, 3, 1, 1],
       [5, 5, 5, 5, 5, 2, 2],
       [1, 1, 1, 1, 1, 7, 5],
       [2, 2, 2, 2, 2, 0, 0],
       [8, 6, 7, 7, 7, 5, 4],
       [7, 8, 6, 6, 6, 4, 3],
       [6, 7, 8, 8, 8, 6, 8],
       [4, 4, 4, 4, 4, 8, 7],
       [0, 0, 0, 0, 0, 3, 6]])

### evluate the rank

In [25]:
# 
def eval_model(name, trueth, pred, startlap = 5):
    match_mask = (trueth == pred)
    print('eval on model: ', name)
    print('accuracy:', np.sum(match_mask)/(match_mask.shape[1]*match_mask.shape[0]))

    top1 = np.logical_and((trueth==0),match_mask)[:,startlap:]
    print('top1 accuracy:', np.sum(top1)/top1.shape[1])
    
    top5 = np.logical_and((trueth < 5),match_mask)[:,startlap:]
    print('top5 accuracy:', np.sum(top5)/top5.shape[1]/5)
    # precision is more useful
    top5 = np.logical_and((trueth < 5),(pred < 5))[:,startlap:]
    print('top5 precision:', np.sum(top5)/top5.shape[1]/5)
    
    
eval_model('lstmts', true_rank, pred_rank)
eval_model('simple', true_rank, simple_rank)




eval on model:  lstmts
accuracy: 0.7103372028745163
top1 accuracy: 0.826530612244898
top5 accuracy: 0.7510204081632653
top5 precision: 0.9102040816326531
eval on model:  simple
accuracy: 0.8413488114980652
top1 accuracy: 0.923469387755102
top5 accuracy: 0.8581632653061224
top5 precision: 0.9561224489795919
